In [3]:
import numpy as np
from math import e
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
import cvxopt as cvo
import copy
from cvxopt import matrix, solvers

In [170]:
class Line():
    def create_line(self):
        x1,x2 = np.random.uniform(-1,1,2)
        y1,y2 = np.random.uniform(-1,1,2)
        self.slope = (y1-y2)/(x1-x2)
        self.b = y1-self.slope*x1
        
    def calculate(self,x):
        return self.slope*x+self.b
    
    def find_actual_y(self,X):
        return [np.sign(y - self.calculate(x)) for x,y in X] #higher than line = 1

In [171]:
class SVM:
    def __init__(self,hard_margin=False,kernel=False,C=.01,gamma=1):
        self.kernel = kernel
        
        self.hard_margin = hard_margin
        self.soft_margin = not self.hard_margin # hard margin with C --> C is huge!
        
        if self.soft_margin:
            self.C = C
        if self.kernel == "poly":
            self.Q = 2
        if self.kernel == "rbf":
            self.gamma = gamma
        
        
    def getQuadCoefs(self):
        if self.kernel == False: #no z space N*N
            print("Kernel=False")
            return np.matmul(self.X,self.X.T)
        else:
            print("Kernel=%s" % self.kernel)
            if self.kernel=="rbf":print("Gamma is %s" %self.gamma)
            if self.kernel=="poly":print("Q is %s" %self.Q)
            return np.array([[self.getKernel(xn,xm) for xm in self.X] for xn in self.X])
        
        
    def getConstraints(self):
        if self.soft_margin:
            print("Soft Margin")
            #-alpha <= 0
            G1 = np.multiply(-1, np.eye(self.N))
            h1 = np.zeros(self.N)
            #alpha <= C
            G2 = np.eye(self.N)
            h2 = np.multiply(np.ones(self.N), self.C)
            
            G = cvo.matrix(np.vstack((G1, G2)))
            h = cvo.matrix(np.hstack((h1, h2)))
            
        if self.hard_margin:
            print("Hard Margin")
            #-alpha <= 0
            G = cvo.matrix(np.multiply(-1, np.eye(self.N)))
            h = cvo.matrix(np.zeros(self.N))
        return G, h
    
    def getAlphas(self):
        cvo.solvers.options['show_progress'] = False
        q = cvo.matrix(np.multiply(-1, np.ones((self.N,1))))
        K = self.getQuadCoefs()
        P = cvo.matrix(np.multiply(np.outer(self.y, self.y), K))
        A = cvo.matrix(self.y.reshape(1, -1), tc='d')
        b = cvo.matrix(0.0)
        G,h = self.getConstraints()
        cvo_sol = cvo.solvers.qp(P,q,G,h,A,b)
        self.alphas = np.ravel(cvo_sol['x'])
        
        
    def get_svs(self):
        return [idx for idx,an in enumerate(self.alphas) if an > 10**-5]
   
    def find_b(self):
        idx = np.argmax(self.alphas)
        xm = self.X[idx]
        ym = self.y[idx]
        if self.kernel != False:
            kernel = [self.getKernel(xn,xm) for xn in self.X]
            ay = np.multiply(self.alphas,self.y)
            return ym-np.dot(ay,kernel)
        else:
            return (1-ym*np.matmul(self.weights.T,xm))/ym
    
    def getKernel(self,x1,x2):
        if self.kernel == "poly":
            return (1+np.matmul(x1,x2.T))**self.Q #POLYNOMIAL
        if self.kernel == "rbf":
            return np.exp(-1 * self.gamma * np.linalg.norm(x1 - x2, ord=2)) #RBF
        
        
        
    
    def calculate_pred(self,x):
        if self.kernel != False:
            ay = np.multiply(self.alphas,self.y)
            kernel = [self.getKernel(xn,x) for xn in self.X]#equals zn*z
            return np.sign(np.dot(ay,kernel))
        else:
            return np.sign(np.dot(self.weights,[1]+x))
            
    def findWeights(self):
        if self.kernel == False:
            self.weights = np.matmul(np.multiply(self.alphas,self.y),self.X)
            self.weights = np.array([self.find_b()] + list(self.weights))
            
    def fit(self,X,y):
        self.X = X
        self.y = y
        self.N = len(self.X)
        
        self.getAlphas()
        print(self.alphas)
        
        self.svs = self.get_svs()
        self.numSVs = len(self.svs)
        self.findWeights()
        
        
    def add_thresholdCol(self,X):
        return np.concatenate([[[1]for x in range(len(X))],X],axis=1)  
    
    def calculate_preds(self,X):
        if self.kernel != False:
            ay = np.multiply(self.alphas,self.y)
            kernel = [[self.getKernel(xn,xm) for xm in X] for xn in self.X]
            return np.sign(np.matmul(ay,kernel))
            
        else:
            X = self.add_thresholdCol(X)
            return np.sign(np.matmul(X, self.weights.T))
        #return np.array([self.calculate_pred(x)for x in X])

In [172]:


        
class create_test(Line):
    def __init__(self,N,hard_margin=True,kernel=False,C=.01,gamma=1):
        self.create_line()

        self.N = N
        self.X = np.random.uniform(-1.0,1.0,(self.N,2))
        self.y = np.array(self.find_actual_y(self.X))
        
        #self.X = np.array([[1,0],[0,1],[0,-1],[-1,0],[0,2],[0,-2],[-2,0]])
        #self.y = np.array([-1,-1,-1,1,1,1,1])#if you use this bc it is w2=0 then it breaks down
        
        
        my_svm = SVM(hard_margin=hard_margin,kernel=kernel,C=C,gamma=gamma)
        my_svm.fit(self.X,self.y)
        
        self.Ein = self.Ein_error(my_svm)
        self.Eout = self.Eout_error(my_svm)
        self.numSvs = my_svm.numSVs
        self.Print()
        
    def Print(self):
        print("SVM Ein Error %s" % self.Ein)
        print("SVM Eout Error %s" % self.Eout)
        print("Num SVs %s" % self.numSvs)
        
    def add_thresholdCol(self,X):
        return np.concatenate([[[1]for x in range(len(X))],X],axis=1)  
        
    def Ein_error(self,my_svm):
        preds = my_svm.calculate_preds(self.X)
        return np.count_nonzero(preds!=self.y)/self.N
    
    def Eout_error(self,my_svm):
        X = np.random.uniform(-1.0,1.0,(1000,2))
        y = self.find_actual_y(X)
        preds = my_svm.calculate_preds(X)
        return np.count_nonzero(preds!=y)/1000

In [178]:
create_test(N=7,hard_margin=True,kernel="rbf")

Kernel=rbf
Gamma is 1
Hard Margin
[0.24608965 1.1193787  3.91181469 2.05072482 0.77009823 3.22153133
 2.14563807]
SVM Ein Error 0.0
SVM Eout Error 0.069
Num SVs 7


In [125]:
#             X = np.concatenate([np.ones((len(self.X),1)),self.X],axis=1)
#             XT = np.concatenate([np.ones((1,len(self.X))),self.X.T],axis=0)
#             return np.matmul(X,XT)**self.Q